In [107]:
import bsscraper
import importlib
importlib.reload(bsscraper)

<module 'bsscraper' from '/Users/stevennewton/Documents/GitHub/MLB_Statcast/notebooka/bsscraper.py'>

In [ ]:
# Database for two years and two teams
bsscraper.make_database('NYY_NYM_19_20.db', [2019,2020], ['NYY,NYM'])

Now I have all statcast data for every pitch thrown by the 2019 and 2020 New York Yankees and New York Mets in a database file. In the next step, I pick out interesting columns and decide to only include 2020 data. I also exclude empty columns which are deprecated stats. In 2020, Major League Baseball switched to a Hawk-Eye tracking system because it was slightly more accurate than Trackman. I'll still have more than enough observations for an analysis if I use 2020 by itself. (Lower down, I accidently deleted events because most of the values are null as they should be so i had to reload the database. It only takes .9 seconds.)

In [261]:
import sqlite3
import pandas as pd
import time

t0  = time.time()
# Connect to, query, and load database into dataframe.
conn = sqlite3.connect('NYY_NYM_19_20_.db')
df = pd.read_sql_query('''
                        SELECT player_name, pitcher, game_year, game_date, game_type, home_team, away_team, inning, pitch_type,
                        pitch_name, pitch_number, release_speed, release_spin_rate, vx0, vy0, vz0, effective_speed, events, 
                        type, description, balls, strikes, zone, release_pos_x, release_pos_z, pfx_x, pfx_z, plate_x, plate_z, 
                        release_extension, ax, ay, az
                        FROM statcast
                        WHERE pitch_type IS NOT NULL and game_year IS 2020;
                       '''
                       , conn)
conn.close()

print(f'{time.time()-t0} seconds!')
df

1.7755157947540283 seconds!


,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,inning,pitch_type,pitch_name,...,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,release_extension,ax,ay,az
0,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,SL,Slider,...,0.76,6.07,-1.07,0.03,0.22,3.29,7.0,-11.171542,29.022734,-32.095575
1,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,SL,Slider,...,0.63,6.03,-0.93,0.31,-0.66,2.26,7.4,-9.423957,27.346872,-28.442330
2,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,0.63,5.99,0.69,1.45,-0.31,2.36,7.3,10.198890,34.624279,-11.155418
3,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,0.48,5.97,0.42,1.50,-0.93,2.65,7.2,6.877947,33.639729,-10.192959
4,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,0.63,5.94,0.48,1.54,0.37,3.60,7.3,6.900246,34.172155,-10.546531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18611,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-1.08,5.53,-0.31,1.33,1.56,3.31,7.0,-6.125120,31.063964,-12.569235
18612,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-1.22,5.51,-0.74,1.39,-0.42,2.72,6.9,-11.223951,35.264626,-11.820192
18613,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-1.15,5.50,-0.61,1.28,1.56,3.34,7.0,-10.585942,32.953592,-13.606505
18614,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-1.23,5.48,-0.59,1.32,0.82,2.41,7.2,-9.820061,32.272418,-12.570117


In [112]:
#To verify this has every pitch, I'll estimate how many rows I expect for 2 teams in 2020.
est_pitches_per_game = 150
games = (2*60)+7   # shortened 2020 season plus postseason
f'I expected approximately {est_pitches_per_game*games} pitches'

'I expected approximately 19050 pitches'

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18616 entries, 0 to 18615
Data columns (total 32 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   player_name        18616 non-null  object 
 1   pitcher            18616 non-null  int64  
 2   game_year          18616 non-null  int64  
 3   game_date          18616 non-null  object 
 4   game_type          18616 non-null  object 
 5   home_team          18616 non-null  object 
 6   away_team          18616 non-null  object 
 7   inning             18616 non-null  int64  
 8   pitch_type         18616 non-null  object 
 9   pitch_name         18616 non-null  object 
 10  pitch_number       18616 non-null  int64  
 11  release_speed      18616 non-null  float64
 12  release_spin_rate  18615 non-null  float64
 13  vx0                18616 non-null  float64
 14  vy0                18616 non-null  float64
 15  vz0                18616 non-null  float64
 16  effective_speed    186

In [156]:
df['pitch_type'].unique()

array(['SL', 'FF', 'FS', 'SI', 'KC', 'CH', 'CU', 'FC', 'KN', 'CS'],
      dtype=object)

In [157]:
df['pitch_name'].unique()

array(['Slider', '4-Seam Fastball', 'Split-Finger', 'Sinker',
       'Knuckle Curve', 'Changeup', 'Curveball', 'Cutter', 'Knuckleball'],
      dtype=object)

I know what all the codes are except CS. Apparently it's a single curveball Michael King threw August 30. I change the CS to CU. Then I'm going to make pitch types into a sparse array for easier grouping. This is usually as a pre-processing step for machine learning algorithms called one-hot encoding. It makes categorical variables numeric.

In [158]:
df[df['pitch_type'] =='CS']

,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,inning,pitch_type,pitch_name,...,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,release_extension,ax,ay,az
5489,Michael King,650633,2020,2020-08-30,R,NYY,NYM,3,CS,Curveball,...,-1.41,6.07,-0.03,1.5,0.22,6.37,7.0,-0.508284,11.843688,-26.880928


In [159]:
df.at[5489, 'pitch_type'] = 'CU'

In [258]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

enc = OneHotEncoder(dtype=int, sparse=False)
x = np.array(df['pitch_type']).reshape(-1, 1)

#Encoder outputs a 2d array. I reattach the header and put the array into a datframe.
arr = enc.fit_transform(x)
arr = np.concatenate((enc.categories_, arr))
arr = pd.DataFrame(arr[1:], columns=arr[0])
arr

,CH,CU,FC,FF,FS,KC,KN,SI,SL
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
18611,0,0,0,1,0,0,0,0,0
18612,0,0,0,1,0,0,0,0,0
18613,0,0,0,1,0,0,0,0,0
18614,0,0,0,1,0,0,0,0,0


In [257]:
pd.concat([df,arr], axis=1)

,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,inning,pitch_type,pitch_name,...,az,CH,CU,FC,FF,FS,KC,KN,SI,SL
0,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,SL,Slider,...,-32.095575,0,0,0,0,0,0,0,0,1
1,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,SL,Slider,...,-28.442330,0,0,0,0,0,0,0,0,1
2,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,-11.155418,0,0,0,1,0,0,0,0,0
3,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,-10.192959,0,0,0,1,0,0,0,0,0
4,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,-10.546531,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18611,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-12.569235,0,0,0,1,0,0,0,0,0
18612,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-11.820192,0,0,0,1,0,0,0,0,0
18613,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-13.606505,0,0,0,1,0,0,0,0,0
18614,Jacob deGrom,594798,2020,2020-07-24,R,NYM,ATL,1,FF,4-Seam Fastball,...,-12.570117,0,0,0,1,0,0,0,0,0


In [259]:
df.head()

,player_name,pitcher,game_year,game_date,game_type,home_team,away_team,inning,pitch_type,pitch_name,...,release_pos_x,release_pos_z,pfx_x,pfx_z,plate_x,plate_z,release_extension,ax,ay,az
0,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,SL,Slider,...,0.76,6.07,-1.07,0.03,0.22,3.29,7.0,-11.171542,29.022734,-32.095575
1,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,SL,Slider,...,0.63,6.03,-0.93,0.31,-0.66,2.26,7.4,-9.423957,27.346872,-28.442330
2,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,0.63,5.99,0.69,1.45,-0.31,2.36,7.3,10.198890,34.624279,-11.155418
3,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,0.48,5.97,0.42,1.50,-0.93,2.65,7.2,6.877947,33.639729,-10.192959
4,Aroldis Chapman,547973,2020,2020-10-09,D,TB,NYY,8,FF,4-Seam Fastball,...,0.63,5.94,0.48,1.54,0.37,3.60,7.3,6.900246,34.172155,-10.546531
